In [1]:
#Initial environement of computations


import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests

from kanon.units import Sexagesimal, Historical, IntegerAndSexagesimal, Temporal
from kanon.calendars import*

Cal=julian_ad = Calendar.registry["Julian A.D."]

number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(584, with_units=False)
Hist_table


Months,Entries
IntegerAndSexagesimal,Historical
31 ;,"29 ; 06,24,00"
59 ;,"01s 28 ; 12,48,00"
90 ;,"02s 27 ; 19,12,00"
120 ;,"03s 26 ; 25,36,00"
151 ;,"04s 25 ; 32,00,00"
181 ;,"05s 24 ; 38,25,00"
212 ;,"06s 23 ; 44,49,00"
243 ;,"07s 22 ; 51,13,00"
273 ;,"08s 21 ; 57,37,00"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(12):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)

In [4]:
#mean motion parameter from the Parisian Alfonsine Tables
daily_mean_elong=Historical(Sexagesimal("12;11,26,41,37,51,50,39"),8)
daily_mean_sun=Historical(Sexagesimal("0;59,8,19,37,19,13,56"),8)
REVO=Historical(Sexagesimal("6,0;0,0"),8)

#length of the synodic month
synodic_month=REVO/daily_mean_elong
synodic_month=Temporal(synodic_month,7)
synodic_month=synodic_month.decimal
synodic_month=float(synodic_month)


for i in range(12):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=4):

        # Computing the mean sun and mean moon for a given number of synodic years
        months=i+1
        mean_sun_moon=(months*synodic_month*daily_mean_sun)%REVO
                       
        #filling the Test and Delta tables with the final precision
        with set_precision(tmode=TruncatureMode.TRUNC, pmode=2):
            Test_recomp[i][1]=1*mean_sun_moon
            Delta_recomp[i][1]=60*60*(Hist_table[i][1]-Test_recomp[i][1])
Test_recomp

Months,Entries
IntegerAndSexagesimal,Historical
31 ;,"29 ; 06,24"
59 ;,"01s 28 ; 12,48"
90 ;,"02s 27 ; 19,12"
120 ;,"03s 26 ; 25,36"
151 ;,"04s 25 ; 32,00"
181 ;,"05s 24 ; 38,25"
212 ;,"06s 23 ; 44,49"
243 ;,"07s 22 ; 51,13"
273 ;,"08s 21 ; 57,37"


In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(0.0, nan)

In [6]:
B.astype(int).style.background_gradient(axis=None)

,Entries
Months,
31 ;,0
59 ;,0
90 ;,0
120 ;,0
151 ;,0
181 ;,0
212 ;,0
243 ;,0
273 ;,0


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Mean syzygies mean sun months.csv')
B.to_csv('Mean syzygies mean sun months.zip', index=False, compression=compression_opts)